In [1]:
#import libraries

import sqlite3
import pandas as pd
import networkx as nx
from pyvis.network import Network

In [2]:
# Connect to SQL

conn = sqlite3.connect('nfts.sqlite')
cur = conn.cursor()

print(cur)

In [3]:
# Get a list of the 5 biggest projects
#get volume of transfers and sort
# divided the sum to reduce the number

df1 = pd.read_sql_query("""
                            SELECT name, nft_address, SUM(transaction_value/1e18) AS volume
                            FROM transfers 
                            INNER JOIN nfts ON transfers.nft_address = nfts.address
                            Group BY transfers.nft_address
                            ORDER By volume DESC
                            LIMIT 5
                        """, conn)

In [4]:
df1

,name,nft_address,volume
0,Art Blocks,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,229537.813149
1,BoredApeYachtClub,0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D,158814.974378
2,MutantApeYachtClub,0x60E4d786628Fea6478F785A6d7e704777c86a7c6,75306.485782
3,Meebits,0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7,66102.198635
4,Loot,0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7,63781.421267


In [5]:
# nft contract address dictionary
contact_name_dict = dict(zip(df1.nft_address, df1.name))
contact_name_dict

{'0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270': 'Art Blocks',
 '0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D': 'BoredApeYachtClub',
 '0x60E4d786628Fea6478F785A6d7e704777c86a7c6': 'MutantApeYachtClub',
 '0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7': 'Meebits',
 '0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7': 'Loot'}

In [6]:
# address tuple
contracts = tuple(contact_name_dict.keys())
contracts

('0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270',
 '0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D',
 '0x60E4d786628Fea6478F785A6d7e704777c86a7c6',
 '0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7',
 '0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7')

In [7]:
# Get a list of all of the nft project names and addresses
all_project_names = pd.read_sql_query("""
                                        SELECT *
                                        FROM nfts 
                                        LIMIT 100000
                                    """, conn)

In [8]:
# dictionary to map all names to contracts
contract_names_dict_all = dict(zip(all_project_names.address, all_project_names.name))
contract_names_dict_all

{'0xF4ee95274741437636e748DdAc70818B4ED7d043': 'The Doge Pound',
 '0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623': 'BoredApeKennelClub',
 '0x7403AC30DE7309a0bF019cdA8EeC034a5507cbB3': 'Characters',
 '0x511372B44231a31527025a3D273C1dc0a83D77aF': 'Maps Restored',
 '0x2E956Ed3D7337F4Ed4316A6e8F2EdF74BF84bb54': 'Neon Junkies',
 '0x03Ea00B0619e19759eE7ba33E8EB8E914fbF52Ea': 'pLOOT',
 '0x06012c8cf97BEaD5deAe237070F9587f8E7A266d': 'CryptoKitties',
 '0xD81f156bBF7043a22d4cE97C0E8ca11d3f4FB3cC': 'Maps',
 '0xD987C5800EF371844CEaC3D0Ee29E4ff29162d7C': 'SpaceLoot',
 '0xdEcC60000ba66700a009b8F9F7D82676B5cfA88A': 'Settlements',
 '0x9F9c171aFdE4cc6bBF6d38aE4012c83633653B85': 'PGFKs',
 '0x73D6F8A959094E0424802bc8adD670f9A790CD1b': 'Snakes On A Chain',
 '0x9759226B2F8ddEFF81583e244Ef3bd13AAA7e4A1': 'Purrnelopes Country Club',
 '0x9d5eD1b12E2BB47E3bDdCC3638376B792b94530B': 'CyberPunk',
 '0x60E4d786628Fea6478F785A6d7e704777c86a7c6': 'MutantApeYachtClub',
 '0x9d418C2Cae665D877F909a725402EbD3A0742844': 'Fang 

In [9]:
# see what wallets own products in all products
# take the top n wallet owners
top_n_oweners_list = pd.read_sql_query("""
                                        SELECT COUNT(DISTINCT nft_address) AS num_projects, owner
                                        FROM current_owners
                                        WHERE nft_address IN {}
                                        GROUP BY owner
                                        ORDER BY num_projects DESC
                                        LIMIT 3
                                    """.format(contracts), conn)
top_n_oweners_list


,num_projects,owner
0,5,0xeB1c22baACAFac7836f20f684C946228401FF01C
1,5,0xd84f2A1E504F15B413ca9feEa52135F01c5C775A
2,5,0xd35B392B385aCAD197E1b89C69cb7CCA7E762f99


In [10]:
#they own 5 expensive projects each

In [11]:
# create a separate tuple for the more spending owners

owners_tuples = tuple(top_n_oweners_list.owner)
owners_tuples

('0xeB1c22baACAFac7836f20f684C946228401FF01C',
 '0xd84f2A1E504F15B413ca9feEa52135F01c5C775A',
 '0xd35B392B385aCAD197E1b89C69cb7CCA7E762f99')

In [12]:
# Go through these owners wallet and see everything they own

top_projects = pd.read_sql_query("""
                                    SELECT nft_address, COUNT(owner) AS count FROM current_owners
                                    WHERE owner IN {}
                                    GROUP BY nft_address
                                    ORDER BY count DESC
                                    LIMIT 50000
                                    """.format(owners_tuples),conn)
top_projects

,nft_address,count
0,0x1C60841b70821dcA733c9B1a26dBe1a33338bD43,44
1,0x31385d3520bCED94f77AaE104b406994D8F2168C,39
2,0xCcc441ac31f02cD96C153DB6fd5Fe0a2F4e6A68d,21
3,0xA74E199990FF572A320508547Ab7f44EA51e6F28,20
4,0x77a251Ac8A70cf15dd2e80329FA8c464101087b0,20
...,...,...
110,0x15f5Ee2362F686617Ee97C166F8E1b681dda7E8c,1
111,0x15A2d6C2b4B9903C27f50Cb8B32160ab17F186E2,1
112,0x066f2D5ead7951F0d0038C19AffD500B9F02c0e5,1
113,0x05a46f1E545526FB803FF974C790aCeA34D1f2D6,1


In [13]:
top_projects_tuple = tuple(top_projects['nft_address'])

In [14]:
# get all th nfts for those top projects
all_nfts_in_top_projects = pd.read_sql_query("""
                            SELECT * 
                            FROM current_owners
                            WHERE nft_address IN{}
                                                        
                            """.format(top_projects_tuple), conn )
all_nfts_in_top_projects

,nft_address,token_id,owner
0,0x004f5683e183908D0f6b688239e3e2D5bbb066CA,0,0x1b3A5b079A9362d4E470879c2C6237104Bc4E49f
1,0x004f5683e183908D0f6b688239e3e2D5bbb066CA,1,0x1b3A5b079A9362d4E470879c2C6237104Bc4E49f
2,0x004f5683e183908D0f6b688239e3e2D5bbb066CA,10,0x1b3A5b079A9362d4E470879c2C6237104Bc4E49f
3,0x004f5683e183908D0f6b688239e3e2D5bbb066CA,100,0xac85b2C302e26E5a994a88203CaFE3797D1361e4
4,0x004f5683e183908D0f6b688239e3e2D5bbb066CA,1000,0x995403718bCdae7A4E63e6e2583df5B734fEe446
...,...,...,...
1305699,0xfb9e9e7150cCebFe42D58de1989C5283d0EAAB2e,9995,0x0000000000000000000000000000000000000001
1305700,0xfb9e9e7150cCebFe42D58de1989C5283d0EAAB2e,9996,0xEBBBBb20f4546a279BDB5b7693f7120A43482E51
1305701,0xfb9e9e7150cCebFe42D58de1989C5283d0EAAB2e,9997,0x0000000000000000000000000000000000000001
1305702,0xfb9e9e7150cCebFe42D58de1989C5283d0EAAB2e,9998,0xeAB37B9b0B3Ff2D24e70bCf7b15ff574b410c4d5


In [15]:
# create edge table
# This will basically create a graph which will depict how much of two NFT is owned by the same owner(weight = number of same owners)

edge_table = pd.read_sql_query("""
                                    SELECT t1.nft_address AS NFT1, t2.nft_address AS NFT2, COUNT(*) AS COUNT
                                    FROM current_owners AS t1
                                    INNER JOIN current_owners AS t2
                                    ON t1.owner = t2.owner
                                    WHERE t1.owner in {}
                                    AND
                                    NFT1 < NFT2
                                    GROUP BY NFT1, NFT2
                                    HAVING COUNT(*) > 50
                                """.format(owners_tuples), conn)
edge_table

,NFT1,NFT2,COUNT
0,0x004f5683e183908D0f6b688239e3e2D5bbb066CA,0x248dE15E3a0da895f4F82cd24bfF89964e4AA381,200
1,0x004f5683e183908D0f6b688239e3e2D5bbb066CA,0x312d09D1160316A0946503391B3D1bcBC583181b,200
2,0x004f5683e183908D0f6b688239e3e2D5bbb066CA,0x3B3ee1931Dc30C1957379FAc9aba94D1C48a5405,80
3,0x004f5683e183908D0f6b688239e3e2D5bbb066CA,0x49aC61f2202f6A2f108D59E77535337Ea41F6540,60
4,0x004f5683e183908D0f6b688239e3e2D5bbb066CA,0x4b3406a41399c7FD2BA65cbC93697Ad9E7eA61e5,60
...,...,...,...
279,0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D,0xCcc441ac31f02cD96C153DB6fd5Fe0a2F4e6A68d,84
280,0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D,0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7,51
281,0xCcc441ac31f02cD96C153DB6fd5Fe0a2F4e6A68d,0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7,252
282,0xCcc441ac31f02cD96C153DB6fd5Fe0a2F4e6A68d,0xa9e26e17B48BBAA125b62730F0904db741533FdC,210


In [16]:
# Adding the names instead of the contract
edge_table['NFT1'] = edge_table['NFT1'].map(contract_names_dict_all)
edge_table['NFT2'] = edge_table['NFT2'].map(contract_names_dict_all)

In [17]:
# get rid of empty nodes
edge_table = edge_table.dropna()

In [18]:
edge_table

,NFT1,NFT2,COUNT
0,Hall Of Fame Goat Lodge,9021Collective,200
1,Hall Of Fame Goat Lodge,Slumdoge Billionaires,200
2,Hall Of Fame Goat Lodge,FND NFT,80
3,Hall Of Fame Goat Lodge,Star Sailor Siblings,60
4,Hall Of Fame Goat Lodge,Lost Poets,60
...,...,...,...
279,BoredApeYachtClub,FLUF,84
280,BoredApeYachtClub,Loot,51
281,FLUF,Loot,252
282,FLUF,WTFoxes,210


In [19]:
# Rename columns
edge_table.columns = ['Source','Target','Weight']

In [20]:
sources = edge_table['Source']
targets = edge_table['Target']
weights = edge_table['Weight']
edge_data = zip(sources, targets, weights)

In [21]:
# Start Visualization
network_graph = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')

In [22]:
# Adding all the partys 
for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    network_graph.add_node(src, src, title=src)
    network_graph.add_node(dst, dst, title=dst)
    network_graph.add_edge(src, dst, value=w)

In [23]:
# add neighbor data to node hover data
neighbor_map = network_graph.get_adj_list()
for node in network_graph.nodes:
    node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])

In [25]:
# display graph
network_graph.toggle_physics(False)
network_graph.show('NFTMap.html')